In [ ]:
import pandas as pd
import numpy as np
######          四季的财报数据            ######
ins=D.instruments(start_date='2016-06-01', end_date='2018-06-30', market='CN_STOCK_A')
finDf=D.financial_statements(instruments=ins ,start_date='2016-06-01',end_date='2018-06-30',      # 净利润         营业总收入          营业总成本                扣除非经常性损益后的净利润                                                                                                  
                          fields=['instrument','fs_quarter','fs_quarter_index','fs_quarter_year','fs_net_income','fs_gross_revenues','fs_total_operating_costs','fs_deducted_profit'])
                                   
beforedf=finDf[finDf.fs_quarter_year==2017]
beforedf=beforedf[beforedf.fs_quarter_index>1]
afterdf=finDf[finDf.fs_quarter_year==2018]
afterdf=afterdf[afterdf.fs_quarter_index<2]
allDf=beforedf.append(afterdf)
allDf.sort_values(by=['instrument'],inplace=True)

allDf.to_csv('aa.csv')

######          去年四季的财报数据            ######
ins=D.instruments(start_date='2014-06-01', end_date='2018-01-30', market='CN_STOCK_A')
befFinDf=D.financial_statements(instruments=ins ,start_date='2016-01-01',end_date='2017-06-30',     # 净利润         营业总收入          营业总成本                扣除非经常性损益后的净利润 
                            fields=['instrument','fs_quarter','fs_quarter_index','fs_quarter_year','fs_net_income','fs_gross_revenues','fs_total_operating_costs','fs_deducted_profit'])                            

bDf=befFinDf[befFinDf.fs_quarter_year==2016]
bDf=bDf[bDf.fs_quarter_index>1]
aDf=befFinDf[befFinDf.fs_quarter_year==2017]
aDf=aDf[aDf.fs_quarter_index<2]
beforeAllDf=bDf.append(aDf)
beforeAllDf.sort_values(by=['instrument'],inplace=True)
beforeAllDf.to_csv('bb.csv')

######          添加字段           ######
allDf['jing_li_run_lv']=np.nan         
allDf['mao_li_run']=np.nan
allDf['mao_li_run_lv']=np.nan
allDf['jing_li_run_kou_chu_lv']=np.nan
allDf['jing_li_run_tong_bi']=np.nan
allDf['mao_li_run_tong_bi']=np.nan
allDf['jing_li_run_kou_chu_tong_bi']=np.nan

count=0                                  ##instrument      fs_quarter   fs_quarter_index    fs_quarter_year  净利润      营业总收入      营业总成本       扣除非经常性损益后的净利润  
for i in allDf.values:                   ##['603999.SHA'   '20180331'           1                 2018       3831651.0   138004432.0    137767616.0     827775.4375          nan]
    #####  净利润率   ######
    allDf.iloc[count,8]=(i[4]/i[5])*100
    #####  毛利润     ######
    allDf.iloc[count,9]=i[5]-i[6]
    #####  毛利润率   ######
    allDf.iloc[count,10]=((i[5]-i[6])/i[5])*100
    #####  扣除非经常性损益后的净利润率   #####
    allDf.iloc[count,11]=(i[7]/i[5])*100
                                            ##instrument   fs_quarter   fs_quarter_index    fs_quarter_year     净利润          营业总收入        营业总成本       扣除非经常性损益后的净利润   
                                            ##'000001.SZA'  '20160630'          2                  2016         12291999744.0    54769000448.0     38613000192.0       12293999616.0]
    for j in beforeAllDf.values:
        if j[0]==i[0] and j[2]==i[2]:
            #####  净利润同比增长   ######'{:.5%}'.format()
            allDf.iloc[count,12]=((i[4]-j[4])/j[4])*100
            #####  毛利润同比增长   ######
            allDf.iloc[count,13]=(((i[5]-i[6])-(j[5]-j[6]))/(j[5]-j[6]))*100
            #####  扣除非经常性损益后的净利润同比增长   ######
            allDf.iloc[count,14]=((i[7]-j[7])/i[7])*100
    count=count+1
#     if count == 50:
#         break

######          history数据             ######
hisDf=D.history_data(instruments=ins,start_date='2018-03-30',end_date='2018-03-30',
                     fields=['instrument','name','industry_sw_level1','concept'])

######             表格拼接             ######
res = pd.merge(allDf,hisDf)

######          导出数据             ######
res.drop(['date','fs_gross_revenues', 'fs_total_operating_costs'],axis=1,inplace=True)
res.fillna(value=-1000)

rows = res.iloc[:,0].size
count = divmod(rows,5000)[0]
for i in range(count):
    res[i*5000:(i+1)*5000].to_csv('fourquarter'+str(i+1)+'.csv')
res[count*5000:].to_csv('fourquarter'+str(count+1)+'.csv')
    
# res.to_csv('fourquarter.csv')

# res.to_csv('alldf.csv')


In [ ]:
###################             concept每个公司                   ####################
instruments = D.instruments()
df = D.history_data(instruments=instruments, start_date='2018-06-25',
                    fields=['concept','name']) 
df1 = D.history_data(instruments=instruments, start_date='2018-06-22',end_date='2018-06-24',   #columns [instrument', 'date', 'name', 'concept']
                    fields=['concept','name'])                                                       #value ['000001.SZA' Timestamp('2018-06-25 00:00:00') '平安银行' '可转债;成交主力;基金减持']

df.sort_values(by=['instrument','date'],inplace=True)                                                                                  
adf = df.drop_duplicates(subset=['instrument','concept'],keep='first').reset_index(drop=True)
adf.sort_values(by=['instrument','date'],inplace=True)

addDf = pd.DataFrame(columns=['name','instrument','new_concept','date'])
print(len(df1.index))
count = 0
num = 0
for i in df1.values:                 #'000001.SZA' Timestamp('2018-06-22 00:00:00') '平安银行' '证金概念;沪深港通重仓'
    num = num +1
    indexx=0
    if pd.isnull(i[3]):
        continue
    bstr=i[3].split(';')
    for j in adf.values:
        if j[1]==i[1]:
            indexx = indexx + 1
            if pd.isnull(j[3]):
                continue
            astr=j[3].split(';')
            add = ','.join(set(astr) - set(bstr))
            if add!='':
                bstr.extend(add.split(',')) 
                addv=pd.Series([j[2],j[1],add,j[0]],index=['name','instrument','new_concept','date'])
                addDf.loc[count]=addv
                count = count + 1
            if indexx == 10:
                break
#addDf=addDf.drop_duplicates(subset=['instrument','new_concept'],keep='first').reset_index(drop=True)
addDf.to_csv('all.csv')

In [ ]:
###############             concept整个市场              ################
import pandas as pd
import numpy as np
import datetime
ins=D.instruments(market='CN_STOCK_A')

nowTime=datetime.datetime.today()
befTime=datetime.timedelta(days=30)
befTime=(nowTime-befTime).strftime('%Y-%m-%d')
df_week=D.history_data(instruments=ins, start_date=befTime,
               fields=['concept','instrument','name'])
df_week=df_week.reset_index(drop = True)

nowTime=datetime.datetime.today()
befTime=datetime.timedelta(days=33)
befTime=(nowTime-befTime).strftime('%Y-%m-%d')

df_moban=D.history_data(instruments=ins, start_date=befTime,end_date=befTime,
               fields=['concept','instrument'])

df_moban=df_moban.reset_index(drop = True)

df=[]
for i in range(len(df_moban.index)):
    if not pd.isnull(df_moban.ix[i,'concept']):
        df.append(df_moban.ix[i,'concept'].split(';'))

stocks=[]
concepts=[]
names=[]
dates=[]
for i in range(len(df_week.index)):
    if not pd.isnull(df_week.ix[i,'concept']):
        cc=df_week.ix[i,'concept'].split(';')
        for j in cc:
            if j not in df:
                concepts.append(j)
#                 stocks.append(df_week.ix[i,'instrument'])
#                 names.append(df_week.ix[i,'name'])
                dates.append(df_week.ix[i,'date'])
df_week=pd.DataFrame({'concept':concepts,'date':dates})

df_week= df_week.drop_duplicates(subset=['concept'],keep='first')
# df_week.sort_values(by=['concept'],axis=0,inplace=True)
df_week=df_week.reset_index(drop = True)
df_week.to_csv('df_moths.csv')
